# Texi Guru Kaggle Challange
* Step 1: Data importing

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

##### Reading test data and train data

In [2]:
try: # For kaggle
  train_pd = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv')

  test_pd = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')
  print('running on kaggle')
except: #for local machine 
  print('running on local machine')
  train_pd = pd.read_csv('train.csv')
  test_pd = pd.read_csv('test.csv')
  pass
copy_train_pd = train_pd.copy()
copy_test_pd = test_pd.copy()
train_pd.info()


running on local machine
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               175000 non-null  int64  
 1   tpep_pickup_datetime   175000 non-null  object 
 2   tpep_dropoff_datetime  175000 non-null  object 
 3   passenger_count        168923 non-null  float64
 4   trip_distance          175000 non-null  float64
 5   RatecodeID             168923 non-null  float64
 6   store_and_fwd_flag     168923 non-null  object 
 7   PULocationID           175000 non-null  int64  
 8   DOLocationID           175000 non-null  int64  
 9   payment_type           175000 non-null  object 
 10  extra                  175000 non-null  float64
 11  tip_amount             175000 non-null  float64
 12  tolls_amount           175000 non-null  float64
 13  improvement_surcharge  175000 non-null  float64
 14  total_amoun

##### Computing the null values and missing values from the tables

In [3]:
train_pd.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-28 17:20:21,2023-06-28 16:34:45,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0
1,0,2023-06-29 23:05:01,2023-06-29 22:01:35,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0
2,1,2023-06-30 10:19:31,2023-06-30 11:13:10,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0
3,0,2023-06-29 13:23:09,2023-06-29 14:20:01,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0
4,1,2023-06-29 22:03:32,2023-06-29 22:22:22,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0


In [4]:
train_pd.isna().sum()

VendorID                    0
tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          6077
trip_distance               0
RatecodeID               6077
store_and_fwd_flag       6077
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
congestion_surcharge     6077
Airport_fee              6077
dtype: int64

In [5]:
train_pd = copy_train_pd.copy()

y = train_pd['total_amount']
X = train_pd.drop('total_amount', axis=1) # seprating prediction element from training set
list_of_features = list(train_pd)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()


## Preprocessing section
* selecting the features for imputation
* transform each feature into same scale
* plotting each feature with respect to each one
* finding the corelation between features

In [7]:
print('Null values in the given training data set is:')
x_train.isna().sum()

Null values in the given training data set is:


VendorID                    0
tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          4840
trip_distance               0
RatecodeID               4840
store_and_fwd_flag       4840
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
congestion_surcharge     4840
Airport_fee              4840
dtype: int64

#### Checking unique values of nan elements

In [8]:
x_train_copy.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee
158161,0,2023-06-29 11:48:37,2023-06-29 11:31:22,1.0,1.50,1.0,N,82,23,Credit Card,2.5,3.452268,0.00,1.0,2.5,0.00
171109,1,2023-06-30 11:48:34,2023-06-30 11:37:19,1.0,3.33,1.0,N,125,173,Credit Card,0.0,8.972716,0.00,1.0,2.5,0.00
60248,1,2023-06-29 20:28:13,2023-06-29 21:39:35,3.0,3.49,1.0,N,210,121,Credit Card,1.0,4.745723,0.00,1.0,2.5,0.00
10735,1,2023-06-29 14:51:19,2023-06-29 17:08:54,1.0,9.66,1.0,N,113,91,Cash,5.0,0.303052,6.55,1.0,0.0,1.75
23536,1,2023-06-30 12:45:18,2023-06-30 13:26:42,1.0,1.23,1.0,N,111,3,Credit Card,0.0,6.275703,0.00,1.0,2.5,0.00


In [9]:
print(f'''
unique values of passanger_count is: {x_train['passenger_count'].unique()}\n
unique values of RateCodeId is: {x_train['RatecodeID'].unique()}\n
unique values of store_and_fwd_flag is: {x_train['store_and_fwd_flag'].unique()}\n
unique values of congestion_surcharge is: {x_train['congestion_surcharge'].unique()}\n
unqiue values of airport_fees is: {x_train['Airport_fee'].unique()}\n
''')




unique values of passanger_count is: [ 1.  3.  5.  2.  4. nan  6.  0.  9.  8.]

unique values of RateCodeId is: [ 1.  3.  5. nan 99.  2.  4.]

unique values of store_and_fwd_flag is: ['N' nan 'Y']

unique values of congestion_surcharge is: [ 2.5  0.   nan -2.5]

unqiue values of airport_fees is: [ 0.    1.75   nan -1.75]




#### Seprating number values and categorical variables

### Creating pipelines for data processing

##### converting datetime into date time object

In [10]:
def convert_dt_obj_to_datetime(df:pd.DataFrame, col_name:str):
    df[col_name] = pd.to_datetime(df[col_name])
    df[col_name +'_Year'] = df[col_name].apply(lambda time: time.year)
    df[col_name +'_Month'] = df[col_name].apply(lambda time: time.month)
    df[col_name +'_Day'] = df[col_name].apply(lambda time: time.day)
    df[col_name +'_Hour'] = df[col_name].apply(lambda time: time.hour)
#     df[col_name +'_Minute'] = df[col_name].apply(lambda time: time.minute) # IGNORING MINS

In [11]:
x_train, x_test = x_train_copy.copy(), x_test_copy.copy()
  
convert_dt_obj_to_datetime(x_train, 'tpep_pickup_datetime')
convert_dt_obj_to_datetime(x_test, 'tpep_pickup_datetime')
x_train = x_train.drop('tpep_pickup_datetime', axis= 1)
x_test = x_test.drop('tpep_pickup_datetime', axis= 1)

convert_dt_obj_to_datetime(x_train, 'tpep_dropoff_datetime')
convert_dt_obj_to_datetime(x_test, 'tpep_dropoff_datetime')

x_train = x_train.drop('tpep_dropoff_datetime', axis= 1)
x_test = x_test.drop('tpep_dropoff_datetime', axis= 1)

x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140000 entries, 158161 to 27071
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   VendorID                     140000 non-null  int64  
 1   passenger_count              135160 non-null  float64
 2   trip_distance                140000 non-null  float64
 3   RatecodeID                   135160 non-null  float64
 4   store_and_fwd_flag           135160 non-null  object 
 5   PULocationID                 140000 non-null  int64  
 6   DOLocationID                 140000 non-null  int64  
 7   payment_type                 140000 non-null  object 
 8   extra                        140000 non-null  float64
 9   tip_amount                   140000 non-null  float64
 10  tolls_amount                 140000 non-null  float64
 11  improvement_surcharge        140000 non-null  float64
 12  congestion_surcharge         135160 non-null  float64
 13  


Creating pipeline for remaining variable

In [12]:
congestion_charger_pipe = Pipeline([
    ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 0)),
#     ('standard_scaler', StandardScaler())
])

airport_fee_pipe = Pipeline([
    ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 0)),
#     ('standard_scaler', StandardScaler())
])

rate_code_id_pipe = Pipeline([
     ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 1)),
#     ('standard_scaler', StandardScaler())
])
store_and_fwd_pipe = Pipeline([
    ('simple_immmputer', SimpleImputer(missing_values= np.nan, strategy='constant', fill_value='N')),
    ('one_hot_encoder', OneHotEncoder())
])
payment_type_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder())
])
passanger_count_pipe = Pipeline([
    ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 0)),
#     ('standard_scaler', StandardScaler())
])



creating column transformers

In [13]:
main_pipeline = ColumnTransformer([
    # ('vendor_id',  StandardScaler(), ["VendorID"]),
    ("passanger_count_t", passanger_count_pipe, ["passenger_count"]),
    ('trip_distance_t', StandardScaler(), ['trip_distance']),
    ('rate_code_id', rate_code_id_pipe, ['RatecodeID'] ),
    ('s_nd_f_flag', store_and_fwd_pipe, ['store_and_fwd_flag']),
    ('pu_loc', StandardScaler(), ['PULocationID']),
    ('du_loc', StandardScaler(), ["DOLocationID"]),
    ('payment_t', payment_type_pipe, ["payment_type"]),
    ('extra_t', StandardScaler(), ['extra']),
    ('tip_amoun_t', StandardScaler(), ['tip_amount']),
    ('tolls_amount_t', StandardScaler(), ["tolls_amount"]),
    ('improvement_c', StandardScaler(), ['improvement_surcharge']),
    ('cong_charge', congestion_charger_pipe, ['congestion_surcharge']),
    ('Airport_fee_t', airport_fee_pipe, ['Airport_fee']),
    ('std_scaler', StandardScaler(), [i for i in range(14, 22)])
], remainder= 'passthrough' )
main_pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('passanger_count_t',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['passenger_count']),
                                ('trip_distance_t', StandardScaler(),
                                 ['trip_distance']),
                                ('rate_code_id',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=1,
                                                                strategy='constant'))]),
                                 ['RatecodeI...
                                ('improvement_c', StandardScaler(),
                                 ['improvement_surcharge']),
                                ('cong_charge',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['congestion_surcharge']),
                                ('Airport_fee_t',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['Airport_fee']),
                                ('std_scaler', StandardScaler(),
                                 [14, 15, 16, 17, 18, 19, 20, 21])])

In [14]:
main_pipeline.fit(x_train)
x_train_t = main_pipeline.transform(x_train)
x_train_t = pd.DataFrame(x_train_t)
print(x_train_t.isna().sum())

x_train_t.head()
# x_train.info()


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,-0.009076,1.0,1.0,0.0,-0.665863,-1.442159,0.0,1.0,0.0,...,0.00,0.0,-0.064387,-0.032734,-0.713934,0.0,-0.088048,0.011649,-0.689350,0.0
1,1.0,-0.004932,1.0,1.0,0.0,-0.101108,0.528578,0.0,1.0,0.0,...,0.00,0.0,-0.064387,0.479815,-0.713934,0.0,-0.088048,0.400860,-0.689350,1.0
2,3.0,-0.004570,1.0,1.0,0.0,1.015270,-0.154611,0.0,1.0,0.0,...,0.00,0.0,-0.064387,-0.032734,0.846004,0.0,-0.088048,0.011649,0.990917,1.0
3,1.0,0.009403,1.0,1.0,0.0,-0.258714,-0.548758,1.0,0.0,0.0,...,1.75,0.0,-0.064387,-0.032734,-0.193955,0.0,-0.088048,0.011649,0.318810,1.0
4,1.0,-0.009688,1.0,1.0,0.0,-0.284982,-1.704924,0.0,1.0,0.0,...,0.00,0.0,-0.064387,0.479815,-0.540608,0.0,-0.088048,0.400860,-0.353296,1.0


# checking data using leanear regression

In [15]:
lr = LinearRegression()
lr.fit(X=x_train_t, y= y_train)
print('mse after basic transformation is:', mean_squared_error(y_train, lr.predict(x_train_t)))
print('train score is:', lr.score(x_train_t, y_train))

mse after basic transformation is: 175.0719273431123
train score is: 0.7292194452172913


test score

In [16]:
x_test_t = main_pipeline.transform(x_test)
x_test_t = pd.DataFrame(x_test_t)
x_test_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       35000 non-null  float64
 1   1       35000 non-null  float64
 2   2       35000 non-null  float64
 3   3       35000 non-null  float64
 4   4       35000 non-null  float64
 5   5       35000 non-null  float64
 6   6       35000 non-null  float64
 7   7       35000 non-null  float64
 8   8       35000 non-null  float64
 9   9       35000 non-null  float64
 10  10      35000 non-null  float64
 11  11      35000 non-null  float64
 12  12      35000 non-null  float64
 13  13      35000 non-null  float64
 14  14      35000 non-null  float64
 15  15      35000 non-null  float64
 16  16      35000 non-null  float64
 17  17      35000 non-null  float64
 18  18      35000 non-null  float64
 19  19      35000 non-null  float64
 20  20      35000 non-null  float64
 21  21      35000 non-null  float64
 22

In [17]:
print('test mean squared error is:', mean_squared_error(y_test, lr.predict(x_test_t)))
print('test score is:', lr.score(x_test_t, y_test))

test mean squared error is: 179.06266832004783
test score is: 0.7228018891012175


In [18]:
# sns.pairplot(x_train)

### Submission code

In [19]:
submission = pd.DataFrame(columns= ['ID', "total_amount"])
submission['ID'] = [i for i in range(1, 50001)]
submission['total_amount'] = [i for i in range(50000)]
submission.to_csv('submission.csv', index=False)